## Import Libs

In [1]:
# import os
import pandas as pd
import numpy as np
import datetime as dt
import logging
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch import optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from entities import *

### Setting 

In [3]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [4]:
symbols = ['AAPL', 'TSLA', 'MSFT']
basket = Basket(symbols=symbols)
basket.load_all_assets(freq="1d")

DEBUG:entities.basket:Initialized Asset Basket: ['AAPL', 'TSLA', 'MSFT'] with 0 assets which loaded.
INFO:entities.basket:Starting batch load for 3 symbols...
DEBUG:entities.basket:Attempting to load AAPL...
DEBUG:entities.asset:Initialized Asset: AAPL with 2724 rows.
INFO:entities.basket:Successfully loaded AAPL (2724 rows).
DEBUG:entities.basket:Attempting to load TSLA...
DEBUG:entities.asset:Initialized Asset: TSLA with 2760 rows.
INFO:entities.basket:Successfully loaded TSLA (2760 rows).
DEBUG:entities.basket:Attempting to load MSFT...
DEBUG:entities.asset:Initialized Asset: MSFT with 2724 rows.
INFO:entities.basket:Successfully loaded MSFT (2724 rows).
INFO:entities.basket:Batch load complete. Success: 3/3. Total assets in basket: 3


In [5]:
targets = ["Close", "High"]
for symbol, asset in basket.assets.items():
    pass

asset.data.head(5)

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,39.933060,40.496700,39.745182,39.847661,27913900
2015-01-05,39.565842,39.907440,39.497520,39.599999,39673900
2015-01-06,38.985104,39.924502,38.891163,39.608523,36447900
2015-01-07,39.480434,39.676854,38.848476,39.266934,29114100
2015-01-08,40.641880,40.778520,39.898901,39.924520,29645200


In [6]:
asset.to_returns(log=True, columns=targets)

DEBUG:entities.asset:MSFT converted to Returns (log=True)


In [7]:
StartPrice    = 100;
NumObs        = 504;  # two calendar years of daily returns
NumSim        = 2;
RetIntervals  = 1;    # one trading day
NumAssets     = 5;

# ExpReturn     = [0.0246  0.0189  0.0273  0.0141  0.0311]/100;
# Sigmas        = [0.9509  1.4259  1.5227  1.1062  1.0877]/100;
# Correlations
# ExpCovariance 

In [8]:
asset.data.head(5)

,Low,Open,Volume,Close (Log_Returns),High (Log_Returns)
Date,,,,,
2015-01-05,39.497520,39.599999,39673900,-0.009238,-0.014658
2015-01-06,38.891163,39.608523,36447900,-0.014787,0.000427
2015-01-07,38.848476,39.266934,29114100,0.012626,-0.006222
2015-01-08,39.898901,39.924520,29645200,0.028994,0.027387
2015-01-09,40.052616,40.658956,23944200,-0.008441,0.001465


In [9]:
asset.inverse_returns(keep=False)

DEBUG:entities.asset:MSFT inversed Returns


In [10]:
asset.data.head(5)

,Low,Open,Volume,Close,High
Date,,,,,
2015-01-05,39.497520,39.599999,39673900,39.565842,39.907440
2015-01-06,38.891163,39.608523,36447900,38.985104,39.924502
2015-01-07,38.848476,39.266934,29114100,39.480434,39.676854
2015-01-08,39.898901,39.924520,29645200,40.641880,40.778520
2015-01-09,40.052616,40.658956,23944200,40.300274,40.838295


In [14]:
features = ["Close", "High"]
tensor_asset = asset.to_tensor(features=features)

DEBUG:entities.asset:Asset: MSFT is using cuda device.


In [15]:
tensor_asset.shape

torch.Size([2723, 2])